In [ ]:
import schedule
import time
import subprocess
from datetime import datetime

# Define the notebook schedule: (notebook name, interval type, interval value)
NOTEBOOK_SCHEDULE = [
    ("Macroeconomic.ipynb", "monthly", "00:20"),  # Runs on the 1st of every month at midnight 00:20am
    ("GDP_Interest_Analyze.ipynb", "monthly", "00:250"),  # Runs on the 1st of every month at midnight 00:25am
    ("Stock_universe.ipynb", "daily", "10:00"),    # Runs daily at 10 AM
    ("CryptoTrends.ipynb", "weekly", "Monday 10:00"),  # Runs every Monday at 10 AM
]

def run_notebook(notebook_name):
    """Execute a Jupyter Notebook"""
    print(f"Running {notebook_name} at {datetime.now()}...")
    subprocess.run(["jupyter", "nbconvert", "--execute", "--to", "notebook", "--inplace", notebook_name])
    print(f"Completed execution of {notebook_name}.\n")

# Schedule each notebook at the specified interval
for notebook, interval_type, interval_value in NOTEBOOK_SCHEDULE:
    if interval_type == "daily":
        schedule.every().day.at(interval_value).do(run_notebook, notebook)
    elif interval_type == "weekly":
        day, time_value = interval_value.split()
        getattr(schedule.every(), day.lower()).at(time_value).do(run_notebook, notebook)

# Handle Monthly Scheduling Separately
def check_and_run_monthly():
    """Runs all monthly scheduled notebooks on the 1st of the month"""
    if datetime.now().day == 1:  # If today is the 1st of the month
        for notebook, interval_type, interval_value in NOTEBOOK_SCHEDULE:
            if interval_type == "monthly":
                run_notebook(notebook)

schedule.every().day.at("00:01").do(check_and_run_monthly)  # Check every day at midnight

print("Notebook scheduler started...")

# Keep the scheduler running
while True:
    schedule.run_pending()
    time.sleep(60)  # Check every minute
